# Implementación de Subword embeddings con RNN

In [17]:
!pip install datasets

In [18]:
import torch
import re
from datasets import load_dataset
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.nn.functional import cross_entropy
from sklearn.metrics import accuracy_score, f1_score

stopwords_español = [
    'a', 'ante', 'bajo', 'cabe', 'con', 'contra', 'de', 'desde', 'durante', 'en', 'entre',
    'hacia', 'hasta', 'mediante', 'para', 'por', 'sin', 'sobre', 'tras', 'y', 'o', 'u',
    'pero', 'mas', 'sino', 'si', 'porque', 'cuando', 'antes', 'despues', 'entonces',
    'siempre', 'nunca', 'nadie', 'todos', 'todas', 'como', 'qué', 'que', 'ser', 'es',
    'eres', 'somos', 'soy', 'son', 'fui', 'fue', 'fueron', 'sea', 'estan', 'esta', 'este',
    'estos', 'estas', 'ese', 'esa', 'eso', 'esos', 'esas', 'aqui', 'alli', 'alla', 'ahi',
    'el', 'la', 'los', 'las', 'un', 'uno', 'unos', 'una', 'unas', 'algo', 'alguien', 'algunos',
    'algún', 'alguna', 'algunas', 'tambien', 'muy', 'le', 'lo', 'les',
    'yo', 'tu', 'él', 'ella', 'ellos', 'ellas', 'nosotros', 'vosotros', 'usted', 'ustedes'
]

In [30]:
def main():
    """
    Entrena y evalúa un modelo RNN para clasificación de intenciones en español usando subword embeddings
    """
    # Establecer el uso del GPU
    dispositivo = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("-"*30)

    # Carga y preprocesa los datos
    conjunto_entrenamiento, conjunto_validación, etiquetas_intención = cargar_y_preprocesar_datos()

    # Se construye el vocabulario inicial y vocabulario BPE
    vocabulario_inicial_palabras = construir_vocabulario_inicial_palabras([conjunto_entrenamiento, conjunto_validación], nombre_columna="texto_filtrado")
    vocabulario_subpalabras, fusiones_bpe = construir_vocabulario_bpe(vocabulario_inicial_palabras, num_fusiones=5000, retornar_fusiones=True)
    print(f"Tamaño del vocabulario de subpalabras: {len(vocabulario_subpalabras)}")
    print("-"*30)

    # Crea dataloaders
    cargador_entrenamiento, cargador_validación = crear_cargadores_datos(conjunto_entrenamiento, conjunto_validación, vocabulario_subpalabras, fusiones_bpe, tamaño_lote=32, longitud_maxima=30)

    # Definición del modelo
    num_clases = conjunto_entrenamiento.features['intent'].num_classes
    modelo = ClasificadorRNN(
        tamaño_vocabulario=len(vocabulario_subpalabras),
        dimensión_embedding=256,
        dimensión_oculta=512,
        dimensión_salida=num_clases,
        tipo_rnn='LSTM',
        dropout=0.5
    ).to(dispositivo)

    modelo_entrenado = entrenar_modelo(modelo, cargador_entrenamiento, cargador_validación, dispositivo, epochs=50, tasa_aprendizaje=1e-3)
    print()

    # Evaluación del modelo
    print("Evaluación final en el conjunto de validación:")
    evaluar_modelo(modelo_entrenado, cargador_validación, dispositivo)
    print()

    # Predicción de ejemplo
    texto_prueba = "Reproduce mi lista de música favorita"
    intención_predicha = predecir(modelo_entrenado, texto_prueba, vocabulario_inicial_palabras, vocabulario_subpalabras, fusiones_bpe, dispositivo, etiquetas_intención)
    print(f"Texto: {texto_prueba}")
    print(f"Intención predicha: {intención_predicha}")

In [21]:
def cargar_y_preprocesar_datos():
    """
    Carga el conjunto de datos MASSIVE en español, aplica preprocesamiento y devuelve
    los conjuntos de entrenamiento y validación.

    Returns:
        conjunto_entrenamiento: Conjunto de entrenamiento preprocesado.
        conjunto_validación: Conjunto de validación preprocesado.
        etiquetas_intención: Mapeo de etiquetas de intención.
    """
    # Carga el conjunto de datos MASSIVE en español
    conjunto_datos = load_dataset('AmazonScience/massive', 'es-ES')

    # Une los splits de entrenamiento y validación para tener más datos
    conjunto_combinado = conjunto_datos['train'].train_test_split(test_size=0.2, seed=42)
    conjunto_entrenamiento = conjunto_combinado['train']
    conjunto_validación = conjunto_combinado['test']

    # Mostrar el número de ejemplos
    print(f"Número de ejemplos de entrenamiento: {len(conjunto_entrenamiento)}")
    print(f"Número de ejemplos de validación: {len(conjunto_validación)}")

    # Aplicar el preprocesamiento al conjunto de entrenamiento y validación
    conjunto_entrenamiento = conjunto_entrenamiento.map(lambda x: {"texto_procesado": preprocesar_texto(x["utt"])})
    conjunto_validación = conjunto_validación.map(lambda x: {"texto_procesado": preprocesar_texto(x["utt"])})

    # Filtrar ejemplos vacíos
    conjunto_entrenamiento = conjunto_entrenamiento.filter(lambda x: len(x["texto_procesado"]) > 0)
    conjunto_validación = conjunto_validación.filter(lambda x: len(x["texto_procesado"]) > 0)

    print("Ejemplo de texto preprocesado:", conjunto_entrenamiento[0]["texto_procesado"])

    # Filtrar palabras raras
    conjunto_entrenamiento = filtrar_palabras_raras(conjunto_entrenamiento, nombre_columna="texto_procesado", frecuencia_min=2)
    conjunto_validación = filtrar_palabras_raras(conjunto_validación, nombre_columna="texto_procesado", frecuencia_min=2)

    return conjunto_entrenamiento, conjunto_validación, conjunto_datos['train'].features['intent']

In [22]:

def preprocesar_texto(texto: str) -> list:
    """
    Preprocesa el texto: minúsculas, eliminación de puntuación, tokenización, eliminación de stopwords y pseudo-lemmatización.

    Args:
        texto (str): Texto a preprocesar.

    Returns:
        List[str]: Lista de tokens procesados.
    """
    # 1. Pasar a minúsculas
    texto = texto.lower()

    # 2. Remover puntuación y caracteres especiales
    texto = re.sub(r'[^a-zñáéíóúü0-9\s]', '', texto)

    # 3. Tokenizar el texto en palabras
    tokens = texto.split()

    # 4. Remover stopwords
    tokens = [token for token in tokens if token not in stopwords_español]

    # 5. Procesamiento simplificado de sufijos comunes (pseudo-lemmatización)
    sufijos_comunes = ['es', 'as', 'os', 'is', 'ar', 'er', 'ir', 'ando', 'iendo']
    tokens_procesados = []
    for token in tokens:
        for sufijo in sufijos_comunes:
            if token.endswith(sufijo) and len(token) > len(sufijo) + 2:
                token = token[: -len(sufijo)]
                break
        tokens_procesados.append(token)

    return tokens_procesados

In [23]:

def filtrar_palabras_raras(conjunto_datos, nombre_columna: str = "texto_procesado", frecuencia_min: int = 2):
    """
    Filtra palabras raras del conjunto de datos que aparecen con frecuencia menor a frecuencia_min.

    Args:
        conjunto_datos: Conjunto de datos a procesar.
        nombre_columna (str): Nombre de la columna que contiene los textos tokenizados.
        frecuencia_min (int): Frecuencia mínima para mantener una palabra.

    Returns:
        Conjunto de datos filtrado.
    """
    contador_frecuencias = Counter()
    for ejemplo in conjunto_datos:
        contador_frecuencias.update(ejemplo[nombre_columna])

    def filtrar_palabras(ejemplo):
        palabras_filtradas = [palabra for palabra in ejemplo[nombre_columna] if contador_frecuencias[palabra] >= frecuencia_min]
        return {"texto_filtrado": palabras_filtradas}

    conjunto_datos = conjunto_datos.map(filtrar_palabras)
    conjunto_datos = conjunto_datos.filter(lambda x: len(x["texto_filtrado"]) > 0)
    return conjunto_datos


In [24]:
def construir_vocabulario_inicial_palabras(lista_conjuntos_datos, nombre_columna: str = "texto_filtrado"):
    """
    Construye un vocabulario inicial de palabras a partir de una lista de conjuntos de datos.

    Args:
        lista_conjuntos_datos (list): Lista de conjuntos de datos.
        nombre_columna (str): Nombre de la columna que contiene los textos tokenizados.

    Returns:
        Counter: Vocabulario con frecuencias de palabras.
    """
    vocabulario = Counter()
    for conjunto_datos in lista_conjuntos_datos:
        for ejemplo in conjunto_datos:
            vocabulario.update(ejemplo[nombre_columna])
    return vocabulario

In [25]:
def construir_vocabulario_bpe(frecuencia_palabras: Counter, num_fusiones: int = 5000, retornar_fusiones: bool = False):
    """
    Construye un vocabulario BPE a partir de las frecuencias de palabras.

    Args:
        frecuencia_palabras (Counter): Frecuencias de palabras.
        num_fusiones (int): Número de fusiones BPE a realizar.
        retornar_fusiones (bool): Si se devuelve el diccionario de fusiones.

    Returns:
        dict: Vocabulario de subpalabras.
        dict (opcional): Diccionario de fusiones BPE.
    """
    vocabulario_bpe = {}
    for palabra, freq in frecuencia_palabras.items():
        caracteres = list(palabra) + ['</w>']
        vocabulario_bpe[tuple(caracteres)] = freq

    fusiones = {}
    for i in range(num_fusiones):
        pares = {}
        for palabra, freq in vocabulario_bpe.items():
            simbolos = palabra
            for j in range(len(simbolos) - 1):
                par = (simbolos[j], simbolos[j + 1])
                pares[par] = pares.get(par, 0) + freq

        if not pares:
            break

        mejor_par = max(pares, key=pares.get)
        fusiones[mejor_par] = i

        # Reemplazar pares en el vocabulario
        vocabulario_bpe_nuevo = {}
        for palabra, freq in vocabulario_bpe.items():
            simbolos_palabra = list(palabra)
            idx = 0
            nueva_palabra = []
            while idx < len(simbolos_palabra):
                if idx < len(simbolos_palabra) - 1 and (simbolos_palabra[idx], simbolos_palabra[idx + 1]) == mejor_par:
                    nueva_palabra.append(simbolos_palabra[idx] + simbolos_palabra[idx + 1])
                    idx += 2
                else:
                    nueva_palabra.append(simbolos_palabra[idx])
                    idx += 1
            vocabulario_bpe_nuevo[tuple(nueva_palabra)] = freq
        vocabulario_bpe = vocabulario_bpe_nuevo

    vocabulario_subpalabras = {}
    indice = 0
    for palabra in vocabulario_bpe.keys():
        for simbolo in palabra:
            if simbolo not in vocabulario_subpalabras:
                vocabulario_subpalabras[simbolo] = indice
                indice += 1

    # Añadir el token 'UNK' si no está presente
    if 'UNK' not in vocabulario_subpalabras:
        vocabulario_subpalabras['UNK'] = indice

    if retornar_fusiones:
        return vocabulario_subpalabras, fusiones
    return vocabulario_subpalabras



In [26]:

def codificar_bpe(palabra: str, fusiones: dict, vocabulario_subpalabras: dict) -> list:
    """
    Codifica una palabra en subpalabras usando el vocabulario y fusiones BPE.

    Args:
        palabra (str): Palabra a codificar.
        fusiones (dict): Diccionario de fusiones BPE.
        vocabulario_subpalabras (dict): Vocabulario de subpalabras.

    Returns:
        List[int]: Lista de índices de subpalabras.
    """
    simbolos = list(palabra) + ['</w>']
    while True:
        pares = [(simbolos[i], simbolos[i+1]) for i in range(len(simbolos)-1)]
        if not pares:
            break
        candidatos = [(par, fusiones.get(par, float('inf'))) for par in pares]
        mejor_par = min(candidatos, key=lambda x: x[1])[0]
        if mejor_par not in fusiones:
            break
        # Fusionar el mejor par
        nuevos_simbolos = []
        idx = 0
        while idx < len(simbolos):
            if idx < len(simbolos) - 1 and (simbolos[idx], simbolos[idx+1]) == mejor_par:
                nuevos_simbolos.append(simbolos[idx] + simbolos[idx+1])
                idx += 2
            else:
                nuevos_simbolos.append(simbolos[idx])
                idx += 1
        simbolos = nuevos_simbolos
    # Convertir símbolos a índices
    tokens_codificados = [vocabulario_subpalabras.get(s, vocabulario_subpalabras['UNK']) for s in simbolos]
    return tokens_codificados


In [27]:

class DatasetClasificaciónTexto(Dataset):
    """
    Dataset para clasificación de texto utilizando codificación BPE.
    """
    def __init__(self, textos, etiquetas, vocabulario_subpalabras, fusiones, longitud_maxima: int = 30):
        """
        Inicializa el dataset.

        Args:
            textos (List[List[str]]): Lista de textos tokenizados.
            etiquetas (List[int]): Lista de etiquetas.
            vocabulario_subpalabras (dict): Vocabulario de subpalabras.
            fusiones (dict): Diccionario de fusiones BPE.
            longitud_maxima (int): Longitud máxima de secuencias.
        """
        self.textos = textos
        self.etiquetas = etiquetas
        self.vocabulario_subpalabras = vocabulario_subpalabras
        self.fusiones = fusiones
        self.longitud_maxima = longitud_maxima

    def __len__(self):
        return len(self.textos)

    def __getitem__(self, idx):
        palabras = self.textos[idx]
        indices_subpalabras = []
        for palabra in palabras:
            indices_subpalabras.extend(codificar_bpe(palabra, self.fusiones, self.vocabulario_subpalabras))
        # Truncar o rellenar la secuencia
        if len(indices_subpalabras) > self.longitud_maxima:
            indices_subpalabras = indices_subpalabras[:self.longitud_maxima]
        else:
            indices_subpalabras += [0] * (self.longitud_maxima - len(indices_subpalabras))
        etiqueta = self.etiquetas[idx]
        return {
            "input_ids": torch.tensor(indices_subpalabras, dtype=torch.long),
            "label": torch.tensor(etiqueta, dtype=torch.long)
        }


In [28]:

def crear_cargadores_datos(conjunto_entrenamiento, conjunto_validación, vocabulario_subpalabras, fusiones, tamaño_lote: int = 32, longitud_maxima: int = 30):
    """
    Crea DataLoaders para los conjuntos de entrenamiento y validación.

    Args:
        conjunto_entrenamiento: Conjunto de datos de entrenamiento.
        conjunto_validación: Conjunto de datos de validación.
        vocabulario_subpalabras (dict): Vocabulario de subpalabras.
        fusiones (dict): Diccionario de fusiones BPE.
        tamaño_lote (int): Tamaño de lote.
        longitud_maxima (int): Longitud máxima de secuencias.

    Returns:
        DataLoader: DataLoader de entrenamiento.
        DataLoader: DataLoader de validación.
    """
    textos_entrenamiento = conjunto_entrenamiento['texto_filtrado']
    textos_validación = conjunto_validación['texto_filtrado']
    etiquetas_entrenamiento = conjunto_entrenamiento['intent']  # Utilizar etiquetas originales
    etiquetas_validación = conjunto_validación['intent']

    dataset_entrenamiento = DatasetClasificaciónTexto(textos_entrenamiento, etiquetas_entrenamiento, vocabulario_subpalabras, fusiones, longitud_maxima)
    dataset_validación = DatasetClasificaciónTexto(textos_validación, etiquetas_validación, vocabulario_subpalabras, fusiones, longitud_maxima)

    cargador_entrenamiento = DataLoader(dataset_entrenamiento, batch_size=tamaño_lote, shuffle=True)
    cargador_validación = DataLoader(dataset_validación, batch_size=tamaño_lote, shuffle=False)

    return cargador_entrenamiento, cargador_validación


In [29]:

class ClasificadorRNN(nn.Module):
    """
    Modelo RNN para clasificación de texto.
    """
    def __init__(self, tamaño_vocabulario: int, dimensión_embedding: int, dimensión_oculta: int, dimensión_salida: int, tipo_rnn: str = 'LSTM', dropout: float = 0.5):
        """
        Inicializa el modelo.

        Args:
            tamaño_vocabulario (int): Tamaño del vocabulario.
            dimensión_embedding (int): Dimensión de los embeddings.
            dimensión_oculta (int): Dimensión de las capas ocultas.
            dimensión_salida (int): Número de clases de salida.
            tipo_rnn (str): Tipo de RNN ('LSTM' o 'GRU').
            dropout (float): Tasa de dropout.
        """
        super(ClasificadorRNN, self).__init__()
        self.embedding = nn.Embedding(tamaño_vocabulario, dimensión_embedding)
        if tipo_rnn == 'LSTM':
            self.rnn = nn.LSTM(dimensión_embedding, dimensión_oculta, batch_first=True, bidirectional=True)
        else:
            self.rnn = nn.GRU(dimensión_embedding, dimensión_oculta, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(dimensión_oculta * 2, dimensión_salida)

    def forward(self, secuencias_texto: torch.Tensor) -> torch.Tensor:
        """
        Define el paso hacia adelante del modelo.

        Args:
            secuencias_texto (torch.Tensor): Tensor de secuencias de entrada.

        Returns:
            torch.Tensor: Salidas del modelo.
        """
        embedded = self.embedding(secuencias_texto)
        if isinstance(self.rnn, nn.LSTM):
            output, (hidden, cell) = self.rnn(embedded)
        else:
            output, hidden = self.rnn(embedded)
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        output = self.fc(hidden)
        return output


In [31]:

def entrenar_modelo(modelo, cargador_entrenamiento, cargador_validación, dispositivo, epochs: int = 200, tasa_aprendizaje: float = 1e-3):
    """
    Entrena el modelo.

    Args:
        modelo: Modelo a entrenar.
        cargador_entrenamiento: DataLoader de entrenamiento.
        cargador_validación: DataLoader de validación.
        dispositivo: Dispositivo ('cuda' o 'cpu').
        epochs (int): Número de épocas.
        tasa_aprendizaje (float): Tasa de aprendizaje.

    Returns:
        Modelo entrenado.
    """
    optimizador = optim.AdamW(modelo.parameters(), lr=tasa_aprendizaje, weight_decay=1e-5)
    programador = optim.lr_scheduler.ReduceLROnPlateau(optimizador, 'min', patience=2)

    for epoch in range(epochs):
        modelo.train()
        pérdida_total = 0
        for lote in cargador_entrenamiento:
            optimizador.zero_grad()
            input_ids = lote["input_ids"].to(dispositivo)
            etiquetas = lote["label"].to(dispositivo)
            salidas = modelo(input_ids)
            pérdida = cross_entropy(salidas, etiquetas)
            pérdida.backward()
            optimizador.step()
            pérdida_total += pérdida.item()
        pérdida_promedio_entrenamiento = pérdida_total / len(cargador_entrenamiento)

        # Evaluación en validación
        modelo.eval()
        pérdida_validación = 0
        todas_predicciones = []
        todas_etiquetas = []
        with torch.no_grad():
            for lote in cargador_validación:
                input_ids = lote["input_ids"].to(dispositivo)
                etiquetas = lote["label"].to(dispositivo)
                salidas = modelo(input_ids)
                pérdida = cross_entropy(salidas, etiquetas)
                pérdida_validación += pérdida.item()
                predicciones = salidas.argmax(dim=1).cpu().numpy()
                todas_predicciones.extend(predicciones)
                todas_etiquetas.extend(etiquetas.cpu().numpy())
        pérdida_promedio_validación = pérdida_validación / len(cargador_validación)
        exactitud_validación = accuracy_score(todas_etiquetas, todas_predicciones)
        f1_validación = f1_score(todas_etiquetas, todas_predicciones, average='weighted')

        print(f"Epoch {epoch+1}/{epochs}")
        print(f"Pérdida de entrenamiento: {pérdida_promedio_entrenamiento:.4f}")
        print(f"Pérdida de validación: {pérdida_promedio_validación:.4f}")
        print(f"Exactitud en validación: {exactitud_validación:.4f}")
        print(f"Puntuación F1 en validación: {f1_validación:.4f}")
        print("-"*30)

        programador.step(pérdida_promedio_validación)

    return modelo



In [32]:
def evaluar_modelo(modelo, cargador_datos, dispositivo):
    """
    Evalúa el modelo en un conjunto de datos.

    Args:
        modelo: Modelo a evaluar.
        cargador_datos: DataLoader del conjunto de evaluación.
        dispositivo: Dispositivo ('cuda' o 'cpu').
    """
    modelo.eval()
    pérdida_total = 0
    todas_predicciones = []
    todas_etiquetas = []
    with torch.no_grad():
        for lote in cargador_datos:
            input_ids = lote["input_ids"].to(dispositivo)
            etiquetas = lote["label"].to(dispositivo)
            salidas = modelo(input_ids)
            pérdida = cross_entropy(salidas, etiquetas)
            pérdida_total += pérdida.item()
            predicciones = salidas.argmax(dim=1).cpu().numpy()
            todas_predicciones.extend(predicciones)
            todas_etiquetas.extend(etiquetas.cpu().numpy())
    pérdida_promedio = pérdida_total / len(cargador_datos)
    exactitud = accuracy_score(todas_etiquetas, todas_predicciones)
    f1 = f1_score(todas_etiquetas, todas_predicciones, average='weighted')
    print(f"Pérdida: {pérdida_promedio:.4f}")
    print(f"Exactitud: {exactitud:.4f}")
    print(f"Puntuación F1: {f1:.4f}")

def predecir(modelo, texto, vocabulario_palabras_inicial, vocabulario_subpalabras, fusiones, dispositivo, etiquetas_intención, longitud_maxima=30):
    """
    Predice la intención de un texto dado.

    Args:
        modelo: Modelo entrenado.
        texto (str): Texto de entrada.
        vocabulario_palabras_inicial (dict): Vocabulario inicial de palabras.
        vocabulario_subpalabras (dict): Vocabulario de subpalabras.
        fusiones (dict): Diccionario de fusiones BPE.
        dispositivo: Dispositivo ('cuda' o 'cpu').
        etiquetas_intención: Mapeo de etiquetas de intención (features).
        longitud_maxima (int): Longitud máxima de secuencia.

    Returns:
        str: Intención predicha.
    """
    modelo.eval()
    palabras_procesadas = preprocesar_texto(texto)
    palabras_filtradas = [palabra for palabra in palabras_procesadas if palabra in vocabulario_palabras_inicial]
    if len(palabras_filtradas) == 0:
        return None
    indices_subpalabras = []
    for palabra in palabras_filtradas:
        indices_subpalabras.extend(codificar_bpe(palabra, fusiones, vocabulario_subpalabras))
    if len(indices_subpalabras) > longitud_maxima:
        indices_subpalabras = indices_subpalabras[:longitud_maxima]
    else:
        indices_subpalabras += [0] * (longitud_maxima - len(indices_subpalabras))
    input_ids = torch.tensor([indices_subpalabras], dtype=torch.long).to(dispositivo)
    with torch.no_grad():
        salida = modelo(input_ids)
        etiqueta_predicha = salida.argmax(dim=1).item()
    # Obtener el nombre de la intención utilizando el mapeo del conjunto de datos
    intención = etiquetas_intención.int2str(etiqueta_predicha)
    return intención

if __name__ == "__main__":
    main()

------------------------------
Número de ejemplos de entrenamiento: 9211
Número de ejemplos de validación: 2303


Map:   0%|          | 0/9211 [00:00<?, ? examples/s]

Map:   0%|          | 0/2303 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9211 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2303 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/9206 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9206 [00:00<?, ? examples/s]

Map:   0%|          | 0/2303 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2303 [00:00<?, ? examples/s]

Tamaño del vocabulario de subpalabras: 2700
------------------------------
Epoch 1/50
Pérdida de entrenamiento: 2.0756
Pérdida de validación: 1.2018
Exactitud en validación: 0.6954
Puntuación F1 en validación: 0.6923
------------------------------
Epoch 2/50
Pérdida de entrenamiento: 0.8664
Pérdida de validación: 0.9526
Exactitud en validación: 0.7530
Puntuación F1 en validación: 0.7533
------------------------------
Epoch 3/50
Pérdida de entrenamiento: 0.5056
Pérdida de validación: 0.9163
Exactitud en validación: 0.7600
Puntuación F1 en validación: 0.7633
------------------------------
Epoch 4/50
Pérdida de entrenamiento: 0.3081
Pérdida de validación: 0.9074
Exactitud en validación: 0.7701
Puntuación F1 en validación: 0.7714
------------------------------
Epoch 5/50
Pérdida de entrenamiento: 0.1968
Pérdida de validación: 0.9460
Exactitud en validación: 0.7622
Puntuación F1 en validación: 0.7637
------------------------------
Epoch 6/50
Pérdida de entrenamiento: 0.1329
Pérdida de valid